# IPyTracker
Jupyter Notebook to perform detection, tracking, and analysis.

## Import Section
Feel free to add lines below to suit your needs. Only remove or edit lines if you know what you're doing.

In [ ]:
from ipywidgets import Video, Image, FloatSlider, IntSlider, Button, Output, VBox
from IPython.display import display
from dataclasses import dataclass, fields
%matplotlib inline 
from matplotlib import pyplot as plt
import numpy as np
import time
import os
import cv2

## Single Image Capture
The below cell captures, saves, and displays a single image. You might find this useful for positioning your camera correctly. Alternatively, use a program of your own choice to display a preview window.

In [26]:
# path to write image to
path = 'data/still.png'

# capture image
os.system(f'libcamera-still --width 768 --height 432 -t 1 -e png -o {path} 2>/dev/null')

# display image
img = Image.from_file(path)
img

Image(value=b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x03\x00\x00\x00\x01\xb0\x08\x02\x00\x00\x00\x91\xb6\…

## Video Capture
Below cell captures video, converts it to the mp4 format, and displays it in a loop. Additionally, timestamps are also saved per frame.

In [35]:
# no file extension in path
path = 'data/recording'

# video capture duration
length = 10000

# capture video
os.system('echo "starting video capture"')
os.system(f'libcamera-vid --width 768 --height 432 --denoise cdn_off --framerate 100 --save-pts {path}.pts -t {length} -o {path}.h264 -n 2>/dev/null')
# os.system(f'libcamera-vid --width 768 --height 432 --framerate 100 -t {length} -o {path}.h264 -n 2>/dev/null')

# convert .h264 to .mp4
os.system('echo "starting video conversion"')
os.system(f'ffmpeg -y -i {path}.h264 {path}.mp4 2>/dev/null')

# display video
vid = Video.from_file(f'{path}.mp4')
vid

starting video capture
starting video conversion


Video(value=b'\x00\x00\x00 ftypisom\x00\x00\x02\x00isomiso2avc1mp41\x00\x00\x00\x08free...')

## Single Frame Selection
This cell allows you to select a single frame from your video. This can be very useful in order to finetune your configuration for the circle detection algorithm. Again, use any other tool if this is not your preferred method.
Hint: single-click the slider and use the arrow keys for precise scrolling.

In [36]:
# path to source video
video_path = 'data/recording.mp4'

# path to write selected frame to
frame_path = 'data/frame.png'

# our image byte array
img = np.ndarray((1,1,1))

# calculate number of frames in video
cap = cv2.VideoCapture(video_path)
nr_frames = cap.get(cv2.CAP_PROP_FRAME_COUNT)
print(nr_frames)

# ui elements
slider = FloatSlider(min=0, max=1.0, step=0.005)
button = Button(description='Save Frame')
out = Output()
display(slider, button)

# gets called each time slider value changes
def slider_callback(change):
    global img
    with out:
        out.clear_output()
        frame_nr = int(change['new'] * nr_frames)
        cap.set(cv2.CAP_PROP_POS_FRAMES, frame_nr)
        ret, img = cap.read()
        if ret:
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            plt.axis('off')
            plt.imshow(img)
            plt.tight_layout()
            plt.show()

# gets called each time button gets pressed
def button_callback(_):
    global img
    with out:
        img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
        cv2.imwrite(frame_path, img)
        print('image saved')
    

# register callback functions
slider.observe(slider_callback, names='value')
button.on_click(button_callback)

# display callback output
out

936.0


FloatSlider(value=0.0, max=1.0, step=0.005)

Button(description='Save Frame', style=ButtonStyle())

Output()

## Circle Detection Algorithm
The actual circle detection algorithm. For more info on the meaning of the config fields, please refer to the openCV documentation.
Right now, this only function only returns a marked image. CSV ouput will be added in the future.

In [42]:
# sharpening kernel
kernel = np.array([ \
    [0, -1, 0],     \
    [-1, 5, -1],    \
    [0, -1, 0]      \
])

# hough detection configuration
@dataclass
class HoughConfig:
    min_dist: int = 10
    hi_thresh: int = 50
    acc_thresh: int = 30
    min_radius: int = 10
    max_radius: int = 0
    preprocessing_thresh: int = 80

# circle detector
def track_objects(img, conf):
    # preprocessing
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    blur = cv2.medianBlur(gray, 5)
    sharp = cv2.filter2D(blur, -1, kernel)
    _, thresh = cv2.threshold(sharp, conf.preprocessing_thresh, 255, cv2.THRESH_BINARY)

    # detect circles
    circles = cv2.HoughCircles(                               \
        thresh, cv2.HOUGH_GRADIENT, 1, conf.min_dist,         \
        param1=conf.hi_thresh, param2=conf.acc_thresh,        \
        minRadius=conf.min_radius, maxRadius=conf.max_radius  \
    )
    if circles is None:
        return thresh

    # draw circles
    circles = np.uint16(np.around(circles))
    res = cv2.cvtColor(thresh, cv2.COLOR_GRAY2BGR)
    for i in circles[0,:]:
        cv2.circle(res, (i[0],i[1]), i[2], (0,255,0), 2)
        cv2.circle(res, (i[0],i[1]), 2, (0,0,255), 3)
    return res

## Detection Configuration
Using the sliders from the cell below, you can create a configuration for the detection algorithm to suit your specific needs. Of course, your configuration is restricted to a certain domain using only these 6 parameters. Luckily, openCV provides a lot of documentation and tutorials, so don't be afraid to get your hands dirty!

In [53]:
# path to read selected frame from
frame_path = 'data/frame.png'

# create a default configuration
config = HoughConfig()

# gets called each time a slider value changes
def slider_callback(change):
    global img
    with out:
        out.clear_output()
        # update config
        setattr(config, change.owner.description, change.new)
        
        # apply tracking
        unmarked = cv2.imread(frame_path)
        marked = track_objects(unmarked, config)
        marked = cv2.cvtColor(marked, cv2.COLOR_BGR2RGB)
        
        # show image
        plt.axis('off')
        plt.imshow(marked)
        plt.tight_layout()
        plt.show()

# gets called each time button gets pressed
def button_callback(_):
    pass
        
# ui elements
button = Button(description='Save Configuration')
sliders = []

# one slider for each field in our config
for field in fields(HoughConfig):
    # create slider
    slider = IntSlider(
        value=getattr(config, field.name),
        min=0,
        max=100,  # Adjust max value as needed
        step=1,
        description=field.name
    )
    # Register the callback with each slider
    slider.observe(slider_callback, 'value')
    sliders.append(slider)
    
# register button callback
button.on_click(button_callback)

out = Output()
display(VBox(sliders), button)

# display callback output
out

Button(description='Save Configuration', style=ButtonStyle())

Output()